In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

/home/kenn/anaconda3/envs/tf_base/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def add_layer(inputs, in_size,out_size, n_layer, activation_function=None):
    layer_name = 'layer%s' % n_layer
    # 把所有内容可视化的时候包含在layer节点内
    with tf.name_scope("layer"):
        with tf.name_scope("weights"):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name="W") # 权重矩阵 行=输入维度，列=输出维度（下一层输入维度）
            # 把参数数据总结到直方图中
            tf.summary.histogram(layer_name+"/weights", Weights)
        with tf.name_scope("biases"):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name="b")           # 偏置向量 行=1，列=输出维度
            tf.summary.histogram(layer_name+"/biases", biases)
        with tf.name_scope("Wx_plus_b"):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases              # 其实是xW，如果行列定义反过来，这里可以Wx，xW的比较符合数据储存直觉
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+"/outputs", outputs)
        return outputs

def compute_accuracy(v_xs, v_ys):
    global prediction #先把prediction定义为全局变量
    y_pre = sess.run(prediction, feed_dict={xs:v_xs}) #生成预测值（概率），10分类，所以一个样本是10列概率
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1)) #比较概率最大值的位置和真实标签位置是否一样，一样就是true
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #计算平均正确率
    result = sess.run(accuracy, feed_dict={xs:v_xs, ys:v_ys}) #生成正确率
    return result
#输入神经元：784个，输出神经元10个
#输入层: None行,784列。None表示这个维度不固定，此即样本数
#placeholder里面需要定义dtype，否则报错
#name是可视化的时候显示的节点名称
#name_scope("inputs")把2个input合并为一个输入节点
with tf.name_scope("inputs"):
    xs = tf.placeholder(tf.float32, [None,784], name = "x_input") # 输入数据：None不限制样本数，784=28x28,每个样本784个像素点（特征）
    ys = tf.placeholder(tf.float32, [None,10], name = "y_input") # 输出数据：None不限制样本数，10个输出（10分类问题）

#（输出层）：输入为上一层输出=784，输出1,激活函数为softmax！！
prediction = add_layer(xs, 784, 10, n_layer=1, activation_function=tf.nn.softmax)

#交叉熵损失
with tf.name_scope("corss_entropy"):
    corss_entropy =  tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
    tf.summary.scalar("corss_entropy",corss_entropy)
    
with tf.name_scope("train"):
    train_step = tf. train.GradientDescentOptimizer(0.5).minimize(corss_entropy)
    
sess = tf.Session()
# 将上述可视化合并
merged = tf.summary.merge_all()
# 将以上结构写入文件
writer = tf.summary.FileWriter("logs/classification",sess.graph)
# 终端激活对应环境后执行 tensorboard --logdi'file:///home/kenn/tensorflow_learning/logs'
# localhost:6006看
init = tf.global_variables_initializer()
sess.run(init)
for i in range(1000):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    #通过placeholder定义输入的话，都要用feed_dict载入数据
    sess.run(train_step, feed_dict = {xs:batch_xs, ys:batch_ys})
    if i%50 == 0:
        print(compute_accuracy(mnist.test.images, mnist.test.labels))
        result = sess.run(merged, feed_dict = {xs:batch_xs, ys:batch_ys})
        writer.add_summary(result,i)

0.0511
0.6441
0.7483
0.7908
0.809
0.825
0.8366
0.8457
0.852
0.8487
0.8576
0.8635
0.8634
0.8663
0.869
0.8686
0.8726
0.8746
0.8739
0.8776
